In [10]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
from alpha_vantage.timeseries import TimeSeries
import sys
import os


## For Collections, Manipulation and Visualization of Data
import numpy as np
import pandas as pd
import pandas_datareader as web
from pandas_datareader.data import DataReader
import matplotlib

import backtrader as bt

In [2]:
NVDA = web.get_data_yahoo("NVDA",
                            start = "2020-01-01",
                            end = "2020-10-31")

In [3]:
NVDA

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-12-31,235.679993,230.130005,230.899994,235.300003,5775100.0,234.912384
2020-01-02,239.910004,236.720001,238.750000,239.910004,5938400.0,239.514771
2020-01-03,237.830002,234.100006,235.100006,236.070007,5134600.0,235.681122
2020-01-06,237.270004,231.270004,232.320007,237.059998,6565900.0,236.669479
2020-01-07,241.770004,236.389999,238.199997,239.929993,7871400.0,239.534729
...,...,...,...,...,...,...
2020-10-26,547.710022,521.539978,537.080017,525.650024,8422400.0,525.494751
2020-10-27,537.679993,527.659973,532.239990,535.869995,6263000.0,535.711731
2020-10-28,530.400024,504.359985,527.919983,505.079987,9413000.0,504.930817


In [4]:
import backtrader as bt
class CrossoverStrategy(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(fast_period=12, slow_period=26, signal_period=9)

    def __init__(self):

        self.fast_ma = bt.indicators.EMA(self.data.close, period=self.p.fast_period)
        self.slow_ma = bt.indicators.EMA(self.data.close, period=self.p.slow_period)
        self.macd_line = self.fast_ma - self.slow_ma
        self.signal_line = bt.indicators.EMA(self.macd_line, period=self.p.signal_period)
        self.macd_crossover = bt.indicators.CrossOver(self.macd_line, self.signal_line)

    def next(self):

        if self.macd_crossover > 0:
            self.buy()  # enter long position
        elif self.macd_crossover < 0:
            self.close()  # close long position

In [7]:
TICKER = "NVDA"
STRATEGY_PARAMS = dict(fast_period=12, slow_period=26, signal_period=9)


def run_backtest(plot=True, **strategy_params):

    cerebro = bt.Cerebro()
    cerebro.adddata(
        bt.feeds.PandasData(dataname=web.get_data_yahoo("NVDA",
                            start = "2020-01-01",
                            end = "2020-10-31"), name=TICKER)
    )

    # Remember to set it high enough or the strategy may not
    # be able to trade because of short of cash
    cerebro.broker.setcash(10000.0)

    # Print out the starting conditions
    print(f"Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}")

    # Although we have defined some default params in the strategy,
    # we can override it by passing in keyword arguments here.
    cerebro.addstrategy(CrossoverStrategy, **strategy_params)

    cerebro.addobserver(bt.observers.Trades)
    cerebro.addobserver(bt.observers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio)

    # Let's say that we have 0.25% slippage and commission per trade,
    # that is 0.5% in total for a round trip.
    cerebro.broker.setcommission(commission=0.0025, margin=False)

    # Run over everything
    strats = cerebro.run()

    print(f"Final Portfolio Value:    {cerebro.broker.getvalue():,.2f}")

    if plot:
        cerebro.plot()

In [11]:
run_backtest(plot=True, **STRATEGY_PARAMS)

Starting Portfolio Value: 10,000.00
Final Portfolio Value:    10,076.99


ImportError: cannot import name 'axes' from 'matplotlib' (C:\Users\tanzh\anaconda3\envs\BT4013\lib\site-packages\matplotlib\__init__.py)